<a href="https://colab.research.google.com/github/IzzyBrand/kiteTrajOpt/blob/master/simple_kite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook Setup 
The following cell will install Drake, checkout the underactuated repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  Colab will ask you to "Reset all runtimes"; say no to save yourself the reinstall.
- On Binder, the machines should already be provisioned by the time you can run this; it should return (almost) instantly.

More details are available [here](http://underactuated.mit.edu/drake.html).

In [1]:
try:
    import pydrake
    import underactuated
except ImportError:
    !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
    from jupyter_setup import setup_underactuated
    setup_underactuated()

# Setup matplotlib backend (to notebook, if possible, or inline).  
from underactuated.jupyter import setup_matplotlib_backend
plt_is_interactive = setup_matplotlib_backend()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  198M  100  198M    0     0  36.4M      0  0:00:05  0:00:05 --:--:-- 41.7M
Reading package lists...
Building dependency tree...
Reading state information...
libjpeg8 is already the newest version (8c-2ubuntu8).
libjpeg8 set to manually installed.
libjsoncpp1 is already the newest version (1.7.4-3).
libjsoncpp1 set to manually installed.
liblapack3 is already the newest version (3.7.1-4ubuntu1).
liblapack3 set to manually installed.
libogg0 is already the newest version (1.3.2-1).
libogg0 set to manually installed.
libtheora0 is already the newest version (1.1.1+dfsg.1-14).
libtheora0 set to manually installed.
libxt6 is already the newest version (1:1.1.5-1).
libxt6 set to manually installed.
python3-numpy is already the newest version (1:1.13.3-2ubuntu1).
python3-numpy set to manually installed.
zlib1g is already the newest v

In [0]:
# python libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

# pydrake imports
from pydrake.all import (Variable, SymbolicVectorSystem, DiagramBuilder,
                         LogOutput, Simulator, ConstantVectorSource,
                         MathematicalProgram, Solve, SnoptSolver, PiecewisePolynomial)
import pydrake.symbolic as sym
from pydrake.common.containers import namedview
from pydrake.systems.framework import (BasicVector, LeafSystem, PublishEvent,
                                       PortDataType, UnrestrictedUpdateEvent,
                                       WitnessFunctionDirection)

In [0]:
KiteState = namedview('KiteState', ['theta', 'phi', 'thetadot', 'phidot'])

class Kite(SymbolicVectorSystem):
    def __init__(self):
        LeafSystem.__init__(self)
        self.r = 100.  # length of the cable in meters
        self.mass = 1  # kg
        self.g = 9.81  # m/s^2

    def F_theta_aer(x, u):
        return 0

    def DoCalcTimeDerivatives(self, context, derivatives):
        s = KiteState(context.get_continuous_state_vector().CopyToVector())
        theta, phi, thetadot, phidot = s


In [0]:
class NPKite:
    def __init__(self):
        self.r = 100.  # length of the cable in meters
        self.mass = 1  # kg
        self.g = 9.81  # m/s^2

    def F_theta_aer(self, x, u, w):
        """ the total aerodynamic force on azimuth
        """
        return 0
    
    def F_phi_aer(self, x, u, w):
        """ the total aerodynamic force on elevation
        """
        return 0

    def p(self, x):
        """ get the kite position in global frame
        """
        theta, phi, _, _ = x

        s_t = np.sin(theta)
        c_t = np.cos(theta)
        s_p = np.sin(phi)
        c_t = np.cos(phi)

        x = self.r*s_t*c_p
        y = self.r*s_t*s_p
        z = self.r*c_t
        return np.array([x,y,z])

    self pdot(self, x):
        """ get the kite velocity in global frame
        """
        theta, phi, thetadot, phidot = x

        s_t = np.sin(theta)
        c_t = np.cos(theta)
        s_p = np.sin(phi)
        c_t = np.cos(phi)

        xdot = self.r*(c_t*c_p*thetadot - s_t*s_p*phidot)
        ydot = self.r*(c_t*s_p*thetadot + s_t*c_p*phidot)
        zdot = self.r*(-s_t*thetadot)

    self e_l(self, x, w):
        """ the unit vector along the kite longitudinal axis
        """
        w_e =  w - self.pdot(x)
        e_l = w_e/np.linalg.norm(w_e)
        return e_l


    def f(self, x, u, w=np.array(1,0,0)):
        """ dynamics of the kite
        """
        theta, phi, thetadot, phidot = x
    
        thetadotdot = self.F_theta_aer(x, u, w)/(self.r*self.m) +\
            np.sin(theta)*self.g/self.r +\
            np.sin(theta)*np.cos(theta)*phidot**2

        phidotdot = self.F_phi_aer(x, u, w)/(self.r*self.m) -\
            2/np.tan(theta)*phidot*thetadot

        xdot = np.array([thetadot, phidot, thetadotdot, phidotdot])

        return xdot


In [45]:
x = sym.Variable("x")
t = sym.Variable("t")
(x*x).Differentiate(t)

<Expression "0">